In [ ]:
%matplotlib inline

# load some utilities (for loading MNIST and plotting)
# also imports most Python modules
%run utils.py

# load MNIST training and test data sets
train_dataset, test_dataset = MNIST_datasets()

# tensors with all MNIST images and labels
train_images, train_labels = MNIST()
test_images, test_labels = MNIST(test=True)

# Probabilistic PCA


As before, let $\mathbf{x} \in \mathbb{R}^{784}$ represent a $28 \times 28$-pixel grayscale image and $\mathbf{z} \in \mathbb{R}^2$ be a two-dimensional latent variable.

Instead of regular PCA we know use [probabilistic PCA](https://rss.onlinelibrary.wiley.com/doi/10.1111/1467-9868.00196) as a model for dimensionality reduction and feature extraction, since probabilistic PCA can be altered and extended quite easily and allows a probabilistic interpretation of the encodings, which helps us with generating new MNIST-like images. In our setting, the probabilistic PCA model is given by 
\begin{align*}
  p(\mathbf{x} \,|\, \mathbf{z}) &= \mathcal{N}\left(\mathbf{x}; \mathbf{W}\mathbf{z} + \boldsymbol{\mu}, \sigma^2 \mathbf{I}_{784}\right), \\
  p(\mathbf{z}) &= \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2),
\end{align*}
with parameters $\mathbf{W} \in \mathbb{R}^{784 \times 2}$, $\boldsymbol{\mu} \in \mathbb{R}^{784}$, and $\sigma^2 > 0$.

In Question 3.3 in the lab document, you showed that for the probabilistic PCA model
\begin{equation*}
    p(\mathbf{x}) = \mathcal{N}(\mathbf{x}; \boldsymbol{\mu}, \mathbf{C}),
\end{equation*}
where
\begin{equation*}
    \mathbf{C} = \mathbf{W} \mathbf{W}^\intercal + \sigma^2 \mathbf{I}_{784}.
\end{equation*}
Thus the log-likelihood of i.i.d. data samples $\{\mathbf{x}_1, \ldots, \mathbf{x}_N\}$ is given by
\begin{equation*}
    \log p(\mathbf{x}_1, \ldots, \mathbf{x}_N; \mathbf{W}, \boldsymbol{\mu}, \sigma^2) = - \frac{N}{2}(784 \log{(2\pi)} + \log{|\mathbf{C}|}) - \frac{1}{2} \sum_{n=1}^N {(\mathbf{x}_n - \boldsymbol{\mu})}^\intercal {\mathbf{C}}^{-1} {(\mathbf{x}_n - \boldsymbol{\mu})}.
\end{equation*}

[Tipping and Bishop](https://rss.onlinelibrary.wiley.com/doi/10.1111/1467-9868.00196) actually derived an expression for the maximum likelihood solution of $\mathbf{W}$, $\boldsymbol{\mu}$, and $\sigma^2$. However, in this lab we do not try to obtain the maximum likelihood solution with the algorithms suggested in their original work. Instead, since the log-likelihood is differentiable with respect to the parameters, we will optimize the parameters with gradient descent, similar to the linear regression example in the Jupyter notebook "Introduction to PyTorch".

We try to minimize the cost function
\begin{equation*}
  J(\mathbf{W}, \boldsymbol{\mu}, \sigma^2) = \log{|\mathbf{C}|} + \frac{1}{N} \sum_{n=1}^N {(\mathbf{x}_n - \boldsymbol{\mu})}^\intercal {\mathbf{C}}^{-1} {(\mathbf{x}_n - \boldsymbol{\mu})},
\end{equation*}
where we neglected all additive constant terms of the log-likelihood and scaled it by $N / 2$.

It is computationally much cheaper to work with the
matrix
\begin{equation*}
\mathbf{M} = \mathbf{W}^\intercal \mathbf{W} + \sigma^2 \mathbf{I}_2 \in \mathbb{R}^{2 \times 2}
\end{equation*}
than with the matrix $\mathbf{C} \in \mathbb{R}^{784 \times 784}$.
Actually one can exploit the special structure of the matrix
$\mathbf{C}$ and use the [matrix determinant lemma](https://en.wikipedia.org/wiki/Matrix_determinant_lemma#Generalization) and the [Woodbury matrix identity](https://en.wikipedia.org/wiki/Woodbury_matrix_identity) to show that
\begin{equation*}
      J(\mathbf{W}, \boldsymbol{\mu}, \sigma^2) = 782 \log \sigma^2 +
      \log{|\mathbf{M}|} +
      \frac{1}{N\sigma^2} \sum_{n=1}^N \left(\|\mathbf{x}_n - \boldsymbol{\mu}\|_2^2 -
        {(\mathbf{x}_n - \boldsymbol{\mu})}^\intercal \mathbf{W}
        {\mathbf{M}}^{-1} \mathbf{W}^\intercal
        (\mathbf{x}_n - \boldsymbol{\mu}) \right).
\end{equation*}

We start by implementing the cost function.

## Task 1

The following Python function `cost_function` will be used to evaluate the cost function $J$ for a given data set
\begin{equation*}
\mathbf{X} = \begin{bmatrix} \mathbf{x}_1^\intercal \\ \vdots \\ \mathbf{x}_N^\intercal \end{bmatrix}
\end{equation*}
and paramters $\mathbf{W}$, $\boldsymbol{\mu}$, and $\log \sigma^2$. To enforce $\sigma^2 > 0$, we optimize the real-valued parameter $\log \sigma^2$ instead of $\sigma^2$. Read through and try to understand the existing implementation. Add a final line to the function that computes and returns the value of the cost function by making use of the already computed values and matrices.

*Hint*: Use the PyTorch functions [`torch.pow`](https://pytorch.org/docs/stable/torch.html#torch.pow) for taking the power of each element in a PyTorch tensor, and [`torch.sum`](https://pytorch.org/docs/stable/torch.html#torch.sum) and [`torch.mean`](https://pytorch.org/docs/stable/torch.html#torch.mean) for computing the sum and the mean of a PyTorch tensor (it is possible to specify the dimension along which the operation should be performed).

In [ ]:
def cost_function(X, W, mu, logsigma2):
    # compute matrix Xshifted with rows (x_n - mu)^T
    # note: mu is defined as a row vector
    Xshifted = X - mu
    
    # compute matrix Y with rows (x_n - mu)^T * W
    Y = Xshifted.mm(W)

    # compute matrix M = W^T * W + sigma^2 I
    sigma2 = logsigma2.exp()
    M = W.t().mm(W) + torch.diagflat(sigma2.expand(2))

    # compute the log-determinant of M
    Mlogdet = M.logdet()

    # compute the inverse of M
    Minverse = M.inverse()

    # compute vector C with C[n] = (x_n - mu)^T * W * M^(-1) * W^T * (x_n - mu)
    C = Y.mm(Minverse).mm(Y.t()).diagonal()
    
    # put everything together and compute loss
    return # WRITE YOUR CODE HERE

Note that in particular for higher-dimensional latent spaces one would want to the [Cholesky decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition) of $\mathbf{M}$ to speed up the computation of $\log{|\mathbf{M}|}$ and ${(\mathbf{x}_n - \boldsymbol{\mu})}^\intercal \mathbf{W} {\mathbf{M}}^{-1} \mathbf{W}^\intercal (\mathbf{x}_n - \boldsymbol{\mu})$ (actually, that is exactly how it is implemented in [PyTorch](https://pytorch.org/docs/stable/_modules/torch/distributions/lowrank_multivariate_normal.html)).

We train the model with [Adam](https://arxiv.org/abs/1412.6980), a gradient-based optimization algorithm with adaptive learning rates for different parameters. Instead of evaluating the gradient based on all images in the training data set
in every step, we compute it from a randomly chosen subset of the training data, a so-called minibatch. The main idea is that the gradients computed from a large enough random subset of the data should be roughly similar to the gradient evaluated on the whole data set, but by using a subset of the data set the computation time can be improved.

In PyTorch data loaders are used for iterating through minibatches. The following code snippet creates data loaders of the MNIST training and test data sets that return minibatches of 500 images and their corresponding labels upon iteration. The samples in the training data set are shuffled whereas the samples in the test data set are always returned in the same order.

In [ ]:
# define data loaders
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=500, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=500)

The next code snippet shows a complete implementation of the training procedure.

In [ ]:
import torch.optim as optim

# define the data loaders
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=500, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=500)

# define the initial model parameters
W = torch.randn((784, 2), requires_grad=True)
mu = torch.zeros(1, 784, requires_grad=True)
logsigma2 = torch.zeros(1, requires_grad=True)

# define the optimizer
optimizer = optim.Adam([W, mu, logsigma2], lr=0.01)

# track the training and test loss
training_loss = []
test_loss = []

# optimize parameters for 20 epochs
for i in range(20):

    # for each minibatch
    for x, _ in train_data:

        # evaluate the cost function on the training data set
        loss = cost_function(x, W, mu, logsigma2)

        # update the statistics
        training_loss.append(loss.item())
        test_loss.append(float('nan'))

        # perform backpropagation
        loss.backward()

        # perform a gradient descent step
        optimizer.step()
        
        # reset the gradient information
        optimizer.zero_grad()

    # evaluate the model after every epoch
    with torch.no_grad():

        # evaluate the cost function on the test data set
        accumulated_loss = 0
        for x, _ in test_data:
            loss = cost_function(x, W, mu, logsigma2)
            accumulated_loss += loss.item()
            
        # update the statistics
        test_loss[-1] = accumulated_loss / len(test_data)
            
    print(f"Epoch {i + 1:2d}: training loss {training_loss[-1]: 9.3f}, "
          f"test loss {test_loss[-1]: 9.3f}")
        
# plot the tracked statistics
plt.figure()
iterations = np.arange(1, len(training_loss) + 1)
plt.scatter(iterations, training_loss, label='training loss')
plt.scatter(iterations, test_loss, label='test loss')
plt.legend()
plt.xlabel('iteration')
plt.show()

## Task 2

Read through and try to understand the implementation of the training procedure above. How does it differ from the implementation of the training procedure in the linear regression example? Answer Question 4.3 in the lab instructions.

We inspect the trained model and perform a similar analysis as for regular PCA. Since we do not update the parameters anymore, no gradients have to be computed in the following sections. We can prevent PyTorch from tracking all our computations and building computational graphs by changing the attribute `requires_grad`.

In [ ]:
W.requires_grad = False
mu.requires_grad = False
logsigma2.requires_grad = False

In Question 3.4 in the lab instructions we showed that the distribution of the latent representation $\mathbf{z}$ conditioned on an image $\mathbf{x}$ is also Gaussian and given by
\begin{equation*}
    p(\mathbf{z} \,|\, \mathbf{x}) = \mathcal{N}\left(\mathbf{z}; \mathbf{M}^{-1} \mathbf{W}^\intercal (\mathbf{x} - \boldsymbol{\mu}), \sigma^2 \mathbf{M}^{-1}\right),
\end{equation*}
where
\begin{equation*}
\mathbf{M} = \mathbf{W}^\intercal \mathbf{W} + \sigma^2 \mathbf{I}_2.
\end{equation*}
We can use this result to encode the MNIST images in the lower-dimensional latent space. In contrast to regular PCA there exists not one unique representation of an image in the latent space, but instead each image gives rise to a whole distribution of representations in the latent space. Here we use the mean
\begin{equation*}
\mathbf{M}^{-1} \mathbf{W}^\intercal (\mathbf{x} - \boldsymbol{\mu})
\end{equation*}
of the normal distribution as encoding. Alternatively, one could sample from the normal distribution.

In [ ]:
# compute M = W^T * W + sigma^2 * I
M = W.t().mm(W) + torch.diagflat(logsigma2.exp().expand(2))

# compute the inverse of M
Minv = torch.inverse(M)

# compute encoding of the training images
train_encoding = (train_images - mu).mm(W).mm(Minv)

# compute encoding of the test images
test_encoding = (test_images - mu).mm(W).mm(Minv)

We visualize the distribution of the encodings in the latent space.

In [ ]:
plot_encoding((train_encoding, train_labels), (test_encoding, test_labels))

For each of the digits $0, 1, \ldots, 9$ we compute the average representation in the latent space by taking the mean of the encodings of the MNIST training data set.

In [ ]:
# compute mean encoding
train_mean_encodings = mean_encodings(train_encoding, train_labels)

Let us show where these average representations are located in the latent space.

In [ ]:
plot_encoding((train_encoding, train_labels), (test_encoding, test_labels),
              train_mean_encodings, annotate=True)

We can decode the representations in the latent space, according to the model definition
\begin{align*}
  p(\mathbf{x} \,|\, \mathbf{z}) &= \mathcal{N}\left(\mathbf{x}; \mathbf{W}\mathbf{z} + \boldsymbol{\mu}, \sigma^2 \mathbf{I}_{784}\right), \\
  p(\mathbf{z}) &= \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2).
\end{align*}
Again in contrast to regular PCA, the model provides us with a whole distribution of possible decodings. Analogously to the encoding discussed above, here we take the mean
\begin{equation*}
\overline{\mathbf{x}} = \mathbf{W}\mathbf{z} + \boldsymbol{\mu}
\end{equation*}
of the normal distribution as representative decoding. Of course, alternatively we could sample from the normal distribution defined by the probabilistic PCA model.

## Task 3

Complete the following code snippet by implementing the representative decoding of the `train_mean_encodings`.

*Hint*: It might be easier to work with the equivalent formulation $\overline{\mathbf{x}}^\intercal = \mathbf{z}^\intercal \mathbf{W}^\intercal + \boldsymbol{\mu}^\intercal$. Note that `mu` is defined as the row vector $\boldsymbol{\mu}^\intercal$!

In [ ]:
# compute mean images
train_mean_images = # WRITE YOUR CODE HERE

plot_images(train_mean_images, torch.arange(10))

Next we define a whole grid of points in the latent space that is spanned by the mean encoding for digit "0" and digit "9" in the training data set.

In [ ]:
# compute grid of latent vectors
zgrid = create_grid(train_mean_encodings[0], train_mean_encodings[9])

# visualize it
plot_encoding((train_encoding, train_labels), (test_encoding, test_labels), zgrid)

Again we decode the latent representations and plot the resulting images.

In [ ]:
# compute mean images
xgrid = zgrid.mm(W.t()) + mu

plot_images(xgrid)

As in regular PCA, it is interesting to compare an image $\mathbf{x}$ with its reconstruction $\mathbf{\tilde{x}}$, to see how much information about $\mathbf{x}$ is kept/lost by the reduction of $\mathbf{x}$ to its representation in the latent space. As we discussed above, different encodings and decodings could be considered in the probabilistic settings, leading to different reconstructions. To be consistent with our analysis above, we take the representative decoding of the representative encoding used above as reconstruction.

We compute the reconstructions of the images in the MNIST test data set and plot them alongside the original images. Moreover, again we evaluate the average squared reconstruction error as an objective measure for the quality of the reconstructions. As a side remark, the reconstruction that we use is [not optimal in the squared reconstruction error sense](https://rss.onlinelibrary.wiley.com/doi/10.1111/1467-9868.00196), and hence the average squared reconstruction error could be improved by defining it in a different way.

In [ ]:
# compute reconstruction
test_reconstruction = test_encoding.mm(W.t()) + mu

# compute average squared reconstruction error
sqerr = (test_images - test_reconstruction).pow(2).sum(dim=1).mean()
print(f"Average squared reconstruction error: {sqerr}")

plot_reconstruction(test_images, test_reconstruction, test_labels)

## Task 4

Now we have performed exactly the same analysis for both the regular and the probabilistic PCA. Compare your results and answer Questions 4.4, 4.5, and 4.6 in the lab instructions.

In regular PCA, a priori no distribution of the latent representations is specified by the model. Hence it is difficult to know which encodings might produce reasonably looking images, which makes sampling new encodings and hence new images challenging. In contrast, probabilistic PCA defines the marginal distribution $p(\mathbf{z})= \mathcal{N}(\mathbf{z}; \boldsymbol{0}, \mathbf{I}_2)$. Thus we can generate new MNIST-like images by sampling encodings from this distribution and decoding them.

In [ ]:
z = torch.randn(25, 2)
x = z.mm(W.t()) + mu

plot_images(x)

## Summary

In this part of the lab session, we replaced regular PCA with its probabilistic formulation. Although the PPCA model is more difficult to train than regular PCA, we are can generate new MNIST-like images in a general way now! Unfortunately, to be honest, the generated images do not look great. We will try to improve this in the next part of the lab.